In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection
from keras.models import Sequential
from keras.layers import Dense, Dropout, InputLayer

In [ ]:
# Loading the data
ufc_data = pd.read_csv("UFC-data/ufc-fights-model.csv")
ufc_data = ufc_data.drop(["Unnamed: 0"], axis=1)

# Seeing if the data set is usable
mcg_fights = pd.concat([ufc_data.loc[ufc_data["Name"] == "Conor McGregor"], ufc_data.loc[ufc_data["Name.1"] == "Conor McGregor"]])
mcg_fights

# Which it is :)

In [ ]:
# Apply categorical labeling with hot 1 encoding
# There are 4 stances, so the labels will be an array of 4 items

stance_open_0 = []
stance_orthodox_0 = []
stance_southpaw_0 = []
stance_switch_0 = []

stance_open_1 = []
stance_orthodox_1 = []
stance_southpaw_1 = []
stance_switch_1 = []

for i, row in ufc_data.iterrows():
    if row["Stance"] == "Open Sta":
        stance_open_0.append(1)
        stance_orthodox_0.append(0)
        stance_southpaw_0.append(0)
        stance_switch_0.append(0)
    elif row["Stance"] == "Orthodox":
        stance_open_0.append(0)
        stance_orthodox_0.append(1)
        stance_southpaw_0.append(0)
        stance_switch_0.append(0)
    elif row["Stance"] == "Southpaw":
        stance_open_0.append(0)
        stance_orthodox_0.append(0)
        stance_southpaw_0.append(1)
        stance_switch_0.append(0)
    elif row["Stance"] == "Switch":
        stance_open_0.append(0)
        stance_orthodox_0.append(0)
        stance_southpaw_0.append(0)
        stance_switch_0.append(1)

    if row["Stance.1"] == "Open Sta":
        stance_open_1.append(1)
        stance_orthodox_1.append(0)
        stance_southpaw_1.append(0)
        stance_switch_1.append(0)
    elif row["Stance.1"] == "Orthodox":
        stance_open_1.append(0)
        stance_orthodox_1.append(1)
        stance_southpaw_1.append(0)
        stance_switch_1.append(0)
    elif row["Stance.1"] == "Southpaw":
        stance_open_1.append(0)
        stance_orthodox_1.append(0)
        stance_southpaw_1.append(1)
        stance_switch_1.append(0)
    elif row["Stance.1"] == "Switch":
        stance_open_1.append(0)
        stance_orthodox_1.append(0)
        stance_southpaw_1.append(0)
        stance_switch_1.append(1)

ufc_data["stance_open_0"] = stance_open_0
ufc_data["stance_orthodox_0"] = stance_orthodox_0
ufc_data["stance_southpaw_0"] = stance_southpaw_0
ufc_data["stance_switch_0"] = stance_switch_0

ufc_data["stance_open_1"] = stance_open_1
ufc_data["stance_orthodox_1"] = stance_orthodox_1
ufc_data["stance_southpaw_1"] = stance_southpaw_1
ufc_data["stance_switch_1"] = stance_switch_1

In [ ]:
# Normalizing the data
columns_to_normalize = ufc_data.columns
columns_to_normalize = columns_to_normalize.drop(["Name", "Stance", "DOB", "Name.1", "Stance.1", "DOB.1", "Win"])
print(columns_to_normalize)

ufc_data[columns_to_normalize] = ufc_data[columns_to_normalize].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [ ]:
x = ufc_data[columns_to_normalize].to_numpy()
y = ufc_data["Win"].to_numpy().reshape((-1, 1))

x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.25, random_state=2020)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# Building the model
model = Sequential()
model.add(InputLayer(input_shape=(32,)))
model.add(Dense(128, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))

In [ ]:
# Compiling the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
arr_metrics = model.fit(x_train, y_train, batch_size=32, epochs=100, validation_data=(x_test, y_test))

In [ ]:
evaluation = model.evaluate(x_test, y_test)

In [ ]:
model.save("models/attempt0.h5")

print(arr_metrics.history.keys())

In [ ]:
plt.plot(arr_metrics.history["accuracy"])
plt.plot(arr_metrics.history["val_accuracy"])
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Train", "Val"])
plt.show()

plt.plot(arr_metrics.history["loss"])
plt.plot(arr_metrics.history["val_loss"])
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Traing", "Val"])
plt.show()